In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
conda install face_recognition

error: bad escape \P at position 28

In [2]:
import os
import urllib.request
import tarfile


In [3]:
import cv2

def detect_faces_opencv(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Load OpenCV's pre-trained Haar Cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return image, faces

# Example usage
image_path = '/kaggle/input/labeled-faces-in-the-wild-lfw/lfw/AJ_Cook/AJ_Cook_0001.jpg'  # Replace with an actual path to an image from the LFW dataset
image, faces = detect_faces_opencv(image_path)
cv2.imshow('Detected Faces', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [4]:
conda install face_recognition


error: bad escape \P at position 28

In [ ]:
import cv2
import os
import numpy as np

def prepare_training_data(data_folder_path):
    # List all directories in the data folder
    dirs = os.listdir(data_folder_path)
    
    faces = []
    labels = []

    for dir_name in dirs:
        label = int(dir_name.split('_')[1])  # Assuming folder name is in the format: name_1, name_2, etc.
        subject_dir_path = os.path.join(data_folder_path, dir_name)
        subject_images_names = os.listdir(subject_dir_path)

        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue

            image_path = os.path.join(subject_dir_path, image_name)
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces_rect = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces_rect:
                face = gray[y:y+w, x:x+h]
                faces.append(face)
                labels.append(label)

    return faces, labels

# Prepare training data
faces, labels = prepare_training_data('data/lfw')

# Create LBPH face recognizer and train
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

# Save the trained model
face_recognizer.save('face_recognizer_model.yml')


In [ ]:
def predict(test_img):
    img = test_img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces_rect = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces_rect:
        face = gray[y:y+w, x:x+h]
        label, confidence = face_recognizer.predict(face)
        label_text = "Person " + str(label)
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img, label_text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    return img

# Example usage
test_img_path = '/kaggle/input/labeled-faces-in-the-wild-lfw/lfw/AJ_Cook/AJ_Cook_0001.jpg'  # Replace with an actual path to an image from the LFW dataset
test_img = cv2.imread(test_img_path)
predicted_img = predict(test_img)
cv2.imshow('Face Recognition', predicted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
